In [1]:
from google.cloud import bigquery
project_id="students-group2"
client = bigquery.Client(project=project_id)

In [2]:
# Dataframes 
df_movies = client.query("SELECT * FROM `students-group2.fatma_aziz.movies`").to_dataframe()
df_ratings = client.query("SELECT * FROM `students-group2.fatma_aziz.ratings`").to_dataframe()

In [3]:
# Vérifier s'il y a des doublons 
print("Doublons dans ratings :", df_ratings.duplicated().sum())
print("Doublons dans movies :", df_movies.duplicated().sum())

Doublons dans ratings : 0
Doublons dans movies : 0


In [4]:
df_movies['year'] = df_movies['title'].str.extract(r'\((\d{4})\)')[0].astype('Int64')

In [5]:
df_movies['genres_list'] = df_movies['genres'].str.split('|')

In [6]:
df_movies

,movieId,title,genres,year,genres_list
0,126929,Li'l Quinquin ( ),(no genres listed),<NA>,[(no genres listed)]
1,135460,Pablo (2012),(no genres listed),2012,[(no genres listed)]
2,138863,The Big Broadcast of 1936 (1935),(no genres listed),1935,[(no genres listed)]
3,141305,Round Trip to Heaven (1992),(no genres listed),1992,[(no genres listed)]
4,141472,The 50 Year Argument (2014),(no genres listed),2014,[(no genres listed)]
...,...,...,...,...,...
10324,85896,Tribute to a Bad Man (1956),Western,1956,[Western]
10325,103570,Dead Man's Burden (2012),Western,2012,[Western]
10326,105223,Colorado Territory (1949),Western,1949,[Western]
10327,128360,The Hateful Eight (2015),Western,2015,[Western]


In [7]:
df_ratings

,userId,movieId,rating,timestamp
0,1,204,0.5,1217895786
1,1,256,0.5,1217895764
2,1,277,0.5,1217895772
3,1,719,0.5,1217895799
4,1,45950,0.5,1217897813
...,...,...,...,...
105334,668,93040,5.0,1331051757
105335,668,98154,5.0,1353208964
105336,668,101862,5.0,1373168467
105337,668,106916,5.0,1388740601


In [8]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df_ratings['user_idx'] = user_encoder.fit_transform(df_ratings['userId'])
df_ratings['movie_idx'] = movie_encoder.fit_transform(df_ratings['movieId'])

In [9]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_ratings, test_size=0.2, random_state=42)

In [10]:
# Compter combien de notes chaque utilisateur a fait
user_counts = train_df.groupby('user_idx')['rating'].count()
print(user_counts)

user_idx
0        95
1        25
2        59
3       101
4        56
       ... 
663      55
664     190
665     172
666      58
667    4508
Name: rating, Length: 668, dtype: int64


In [11]:
# Compter combien de notes chaque film a reçu
movie_counts = train_df.groupby('movie_idx')['rating'].count()
print(movie_counts)


movie_idx
0        188
1         69
2         49
3          8
4         53
        ... 
10320      1
10321      1
10322      1
10323      2
10324      1
Name: rating, Length: 9565, dtype: int64


In [28]:
import os
from pathlib import Path
os.chdir('/home/jupyter/Fatma_Aziz/movie_recommender')
print(f"📌 Nouveau dossier: {os.getcwd()}")

📌 Nouveau dossier: /home/jupyter/Fatma_Aziz/movie_recommender


In [30]:
import os

# Chemin relatif vers la racine du projet
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # remonte d'un dossier
data_path = os.path.join(project_root, 'data')

# Créer le dossier data à la racine si pas déjà existant
os.makedirs(data_path, exist_ok=True)

print("✅Le dossier data sera créé ici :", data_path)

✅Le dossier data sera créé ici : /home/jupyter/Fatma_Aziz/data


In [31]:
# Sauvegarder les données dans un dossier data 
df_ratings.to_csv(os.path.join(data_path, 'df_ratings.csv'), index=False)
train_df.to_csv(os.path.join(data_path, 'train_ratings.csv'), index=False)
test_df.to_csv(os.path.join(data_path, 'test_ratings.csv'), index=False)
df_movies.to_csv(os.path.join(data_path, 'movies.csv'), index=False)

In [32]:
df_ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'user_idx', 'movie_idx'], dtype='object')

In [33]:
import pickle
# Sauvegarder les encoders
with open(os.path.join(data_path, 'user_encoder.pkl'), 'wb') as f:
    pickle.dump(user_encoder, f)

with open(os.path.join(data_path, 'movie_encoder.pkl'), 'wb') as f:
    pickle.dump(movie_encoder, f)

print("✅ Encoders sauvegardés dans", data_path)

✅ Encoders sauvegardés dans /home/jupyter/Fatma_Aziz/data
